# 4.4 Logic Programming

## 4.55

```scheme
;; 1
(supervisor ?x (Bitdiddle Ben))

;; 2
(job ?x (accounting . ?type))

;; 3
(address ?x (Sumerville . ?y))
```

## 4.56

```scheme
;; 1
(and (supervisor ?person (Bitdiddle Ben))
     (address ?person . ?street))
;; 2
(and (salary (Bitdiddle Ben) ?x)
     (salary ?person ?amount)
     (lisp-value < ?amount ?x))

;; 3
(and (supervisor ?person ?supervisor)
     (job ?supervisor ?job)
     (not (job ?supervisor (computer . ?type))))
```

## 4.57

```scheme
(rule (can-replace ?person1 ?person2)
      (and
       (or
        ;; person1 does same job as person2
        (and (job ?person1 ?job) 
             (job ?person2 ?job))
        ;; OR someone who does person1's job can also do person2's job
        (and (job ?person1 ?job1)
             (job ?person2 ?job2)
             (can-do-job ?j1 ?j2)))
       ;; person1 and person2 are not the same person
       (not (same ?person1 ?person2))))

;; 1
(can-replace ?person (Fect Cy D))

;; 2 

(and (can-replace ?person1 ?person2)
     (salary ?person1 ?salary1)
     (salary? ?person2 ?salary2)
     (lisp-value > salary2 salary1))
```